In [ ]:
!pip install -qU langchain
!pip install -qU pandas

In [ ]:
%pip install -qU langchain-ollama

In [ ]:
%pip install -U ollama

In [ ]:
#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

from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("GROQ_API_KEY")



In [ ]:
import pandas as pd
from langchain_groq import ChatGroq
from langchain.agents import initialize_agent, AgentType
from langchain.agents import Tool
from langchain.memory import ConversationBufferMemory
import re

# Load CSV file
df = pd.read_csv("forecast.csv")

nepali_months = {
    # Expanded with common variants and misspellings
    "baisakh": "01", "baisak": "01", "baishakh": "01", "baishak": "01",
    "jestha": "02", "jeshtha": "02", "jesth": "02", "jeth": "02",
    "asadh": "03", "asad": "03", "ashad": "03", "ashadh": "03", "asaadh": "03","asar":"03",
    "shrawan": "04", "saun": "04", "shawan": "04", "sawan": "04", "shraban": "04",
    "bhadra": "05", "bhadau": "05", "bhadaw": "05", "bhad": "05",
    "asoj": "06", "ashoj": "06", "ashwin": "06",
    "kartik": "07", "kattik": "07",
    "mangsir": "08", "mangshir": "08", "mansir": "08",
    "poush": "09", "push": "09", "pous": "09", "posh": "09",
    "magh": "10", "mag": "10", 
    "falgun": "11", "phalgun": "11", "falgoon": "11", "fagun": "11",
    "chaitra": "12", "chait": "12", "chait": "12"
}

def query_csv(query):
    """
    This tool extracts forecasted sales data from the dataset based on user queries. 
    It understands both standard date formats (e.g., '2082-04') and Nepali month names (e.g., '2082 Baisakh').
    
    - Example Queries:
      - "What is the sales of 11th month?"
      - "What is the forecasted sales of 2082-01?"
      - "Give me the average sales from 11 to 04."
      - "Give me the average sales of 2082 Baisakh."
      - "What will be the forecasted sales of upcomming Baisakh."
      - "The csv data it is taking is all future date, so any future related keyword can be mapped to these days."
      - "if user ask about relative dates like next|upcoming|previous|last|current at first calculate the present date and identify it accordingly."


    The tool will return either a specific sales value or the average sales over a given range.
    """

    query = query.lower()
    
    match = re.search(r"(\d{4})[-\s]?(\d{2}|\w+)", query)

    
    if match:
        year, month = match.groups()
        
        if month in nepali_months:
            month = nepali_months[month]

        month = month.zfill(2)

        result = df[df["forecast_date"] == f"{year}-{month}"]
        if not result.empty:
            return f"Forecasted sales for {year}-{month}: {result['forecast_sales'].values[0]}"
        else:
            return f"No forecast data found for {year}-{month}"

    elif "average" in query or "mean" in query:
        months = re.findall(r"(\d{2}|\w+)", query)
        numeric_months = []

        for month in months:
            if month in nepali_months:
                numeric_months.append(nepali_months[month])
            elif month.isdigit():
                numeric_months.append(month.zfill(2))
        
        # Filter the dataframe
        filtered_df = df[df["forecast_date"].str[-2:].isin(numeric_months)]
        
        if not filtered_df.empty:
            avg_sales = filtered_df["forecast_sales"].mean()
            return f"Average forecasted sales from {months[0]} to {months[-1]}: {avg_sales:.2f}"
        else:
            return "No data available for the given months."

    return "Sorry, I couldn't understand the query."

llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0,
)

tools = [
    Tool(
        name="CSV Reader",
        func=query_csv,
        description=(
            "Extracts forecasted sales data from the dataset based on user queries. "
            "Understands both numerical months (e.g., '2082-04') and Nepali month names (e.g., '2082 Baisakh') and realtive months"
            "The csv data it is taking is all future date, so any future related keyword can be mapped to these days."
            "if user ask about relative dates like next|upcoming|previous|last|current at first calculate the present date and identify it accordingly."
            "Supports retrieving specific sales or calculating the average sales over a given range."
            "If the user ask about sales in- between the ranges of months then, read and give data seperately for all inbetween dates."
            "Read above Nepali date correctly and get its index properly"
        ),
    )
]

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory,
    verbose=True  
)

agent = initialize_agent(
    tools = tools,
    llm=llm,

    
)

# # Example Queries
# queries = [
#     "What is the sales of 11th month?",
#     "What is the sales of 2082-01?",
#     "What is the forecasted sales of 2082-03?",
#     "Give me the average sales from 11 to 04.",
#     "Give me the average sales of 2082 Baisakh."
# ]

# for q in queries:
#     print(f"Query: {q}")
#     print(f"Response: {agent.run(q)}\n")

while True:
    query = input("Enter the Query")
    response = agent.run(query)
    print(response)

In [ ]:

from langchain.chains import ConversationChain



conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)

In [ ]:
conversation.predict(input="Who old are you?")

### NLP


### Real model

In [2]:
#API KEY for Together AI
# 77a4ea6f1e0798be497502e5f8fcd0f2233b44b3c67080f70a94959ac5ba6ee6
import getpass
import os

if "TOGETHER_API_KEY" not in os.environ:
    os.environ["TOGETHER_API_KEY"] = getpass.getpass("Enter your Together API key: ")

In [3]:
import pandas as pd
from langchain_together import ChatTogether
from langchain.agents import initialize_agent, AgentType
from langchain.agents import Tool
from langchain.memory import ConversationBufferMemory
import re

# Load CSV file
df = pd.read_csv("forecast.csv")

nepali_months = {
    "baisakh": "01", "baisak": "01", "baishakh": "01", "baishak": "01",
    "jestha": "02", "jeshtha": "02", "jesth": "02", "jeth": "02",
    "asadh": "03", "asad": "03", "ashad": "03", "ashadh": "03", "asaadh": "03", "asar": "03",
    "shrawan": "04", "saun": "04", "shawan": "04", "sawan": "04", "shraban": "04",
    "bhadra": "05", "bhadau": "05", "bhadaw": "05", "bhad": "05",
    "asoj": "06", "ashoj": "06", "ashwin": "06",
    "kartik": "07", "kattik": "07",
    "mangsir": "08", "mangshir": "08", "mansir": "08",
    "poush": "09", "push": "09", "pous": "09", "posh": "09",
    "magh": "10", "mag": "10",
    "falgun": "11", "phalgun": "11", "falgoon": "11", "fagun": "11",
    "chaitra": "12", "chait": "12"
}

def query_csv(query):
    query = query.lower()
    
    match = re.search(r"(\d{4})[-\s]?(\d{2}|\w+)", query)

    if match:
        year, month = match.groups()
        
        if month in nepali_months:
            month = nepali_months[month]

        month = month.zfill(2)

        result = df[df["forecast_date"] == f"{year}-{month}"]
        if not result.empty:
            return f"Forecasted sales for {year}-{month}: {result['forecast_sales'].values[0]}"
        else:
            return f"No forecast data found for {year}-{month}"

    elif "average" in query or "mean" in query:
        months = re.findall(r"(\d{2}|\w+)", query)
        numeric_months = []

        for month in months:
            if month in nepali_months:
                numeric_months.append(nepali_months[month])
            elif month.isdigit():
                numeric_months.append(month.zfill(2))
        
        filtered_df = df[df["forecast_date"].str[-2:].isin(numeric_months)]
        
        if not filtered_df.empty:
            avg_sales = filtered_df["forecast_sales"].mean()
            return f"Average forecasted sales from {months[0]} to {months[-1]}: {avg_sales:.2f}"
        else:
            return "No data available for the given months."

    return "Sorry, I couldn't understand the query."

# Use the best free LLM
# llm = ChatTogether(
#     model="deepseek-chat",  # Or use "mistral-7b-instruct"
#     temperature=0,
#     together_api_key="tgp_v1_KOT7FQuHS1-LqTemPcrfY7Ta7Fr7bVaEVUpSvnPDsqE"
# )
llm = ChatTogether(
    model="meta-llama/Llama-3-70b-chat-hf",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    
)

tools = [
    Tool(
        name="CSV Reader",
        func=query_csv,
        description=(
            "Extracts forecasted sales data from the dataset based on user queries. "
            "Understands both numerical months (e.g., '2082-04') and Nepali month names (e.g., '2082 Baisakh') and relative months. "
            "If the user asks about relative dates like next|upcoming|previous|last|current, first calculate the present date and identify it accordingly. "
            "Supports retrieving specific sales or calculating the average sales over a given range. "
            "If the user asks about sales within a range of months, return data separately for all in-between dates. "
            "Read Nepali month names correctly and get their index properly."
            "Give the final answer in json format with date and sales value"
            
        ),
    )
]

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory,
    verbose=True  
)

while True:
    query = input("Enter the Query: ")
    response = agent.run(query)
    print(response)


C:\Users\binay\AppData\Local\Temp\ipykernel_30416\2126064315.py:97: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
C:\Users\binay\AppData\Local\Temp\ipykernel_30416\2126064315.py:99: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = init



> Entering new AgentExecutor chain...
Thought: I need to extract the sales data for the month of Shrawan from the dataset.

Action: CSV Reader
Action Input: "2082 Shrawan"
Observation: Forecasted sales for 2082-04: 480692563.7007759
Thought:I have the sales data for Shrawan.

Final Answer: {"date": "2082-04", "sales": 480692563.7007759}

> Finished chain.
{"date": "2082-04", "sales": 480692563.7007759}


> Entering new AgentExecutor chain...
Thought: I need to extract the sales data from the dataset for the months of Baisakh to Shrawan.

Action: CSV Reader
Action Input: "sales from 2082 Baisakh to 2082 Shrawan"
Observation: Forecasted sales for 2082-01: 1782385514.8297076
Thought:I need to extract the sales data for the remaining months between Baisakh and Shrawan.

Action: CSV Reader
Action Input: "sales from 2082 Jestha to 2082 Ashad"
Observation: Forecasted sales for 2082-02: 1778089645.1841457
Thought:Thought: I need to combine the sales data for all the months from Baisakh to Sh

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `I'm ready to answer your question. Please go ahead and ask your question about forecasted sales data.`
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

In [ ]:
%pip install -qU langchain-together